# Shape analysis

This notebook shows how to perform shape analysis using [GEMDAT](https://gemdat.readthedocs.org). 

This uses a generalized algorithm that for all symmetrically equivalent cluster centers, finds nearest atoms from trajectory, and transforms them back to the asymmetric unit. This helps the statistics for performing shape analysis and making plots.

As input you will need:

1. [symmetrized crystal or material structure](https://pymatgen.org/pymatgen.symmetry.html#pymatgen.symmetry.structure.SymmetrizedStructure)
    - contains unique sites to act as cluster centers (asymmetic unit)
    - provides symmetry operations
    - Gemdat uses [SpacegroupAnalyzer](https://pymatgen.org/pymatgen.symmetry.html#pymatgen.symmetry.analyzer.SpacegroupAnalyzer) to find symmetry in P1 structures
2. [trajectory](https://pymatgen.org/pymatgen.symmetry.html#pymatgen.symmetry.analyzer.SpacegroupAnalyzer)
    - typically P1
    - maybe a supercell of the structure in #1
    - lattice can be triclinic (non-constrained in simulation)

The `ShapeAnalyzer` algorithm works as follows:

1. reduce supercell of trajectory to match clusters
    - assert trajectory and cluster lattices are similar
2. for every unique center:
    - for every symmetry operation:
        - apply symmetry operation to center coordinates
        - find all trajectory points within X distance of transformed coordinates (wrap around pbc)
        - copy and map points back to asymmetric unit (reverse symmetry op)
        - subtract center coords
    - concatenate all coords and convert to Cartesian coordinate system

The result is a set of trajectory positions centered on the input sites. These can be used to perform shape analysis: plots, fits, heat maps, msd, etc.

See: https://github.com/GEMDAT-repos/GEMDAT/pull/166

## Loading data

Loading trajectory for argyrodite and cif file with 3 unique sites as cluster centers.

In [ ]:
from pathlib import Path

from gemdat import Trajectory, load_known_material

from gemdat.utils import VASPRUN

# Use your own data:
# VASPRUN = 'path/to/your/vasprun.xml'

trajectory = Trajectory.from_vasprun(VASPRUN)
diff_trajectory = trajectory.filter('Li')

structure = load_known_material('argyrodite', supercell=(2, 1, 1))

## Set up shape analyzer

Note that the we use the `.from_structure` constructor. This symmetrizes the input structure.

In [ ]:
from gemdat.shape import ShapeAnalyzer

sa = ShapeAnalyzer.from_structure(structure)
sa

## Clustering

The next cell shows how to run the shape analysis. The radius in Ångström is used to determine if an atom belongs to a cluster.

The supercell is used to fold the trajectory coordinates to the same lattice as the input sites. A warning will be raised if the lattices are not close.

Alternatively, you can pass any numpy array with fractional coordinates to `sa.analyze_positions()`

In [ ]:
supercell = (2, 1, 1)
radius = 1.5  # Å

shapes = sa.analyze_trajectory(
    trajectory=diff_trajectory,
    supercell=supercell,
    radius=radius,
)

shapes[0]

## Plot shapes

Shape data can be plotted using the `plots` module. This shows each set of shape coordinates from all three directions.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gemdat.plots.matplotlib as gemplt

bins = np.linspace(-radius, radius, 50)

for shape in shapes:
    fig = shape.plot(nbins=len(bins))
    fig.show()
    
    gemplt.shape(shape=shape,bins=bins)
    plt.show()


## Plot nearby sites

You can supply a nearby sites to display them on the plot.

In [ ]:
shape = shapes[0]

sites = structure.get_sites_in_sphere(pt=shape.origin, r=radius)

fig = shape.plot(nbins=len(bins), sites=sites);
fig.show()
    
gemplt.shape(shape=shape,bins=bins, sites=sites)
plt.show()

## Optimize sites

You can optimize the site positions using the `ShapeAnalyzer.optimize_sites()` method. This uses the centroid of the shapes to shift the sites.

In [ ]:
sa_shifted = sa.optimize_sites(shapes=shapes)
sa_shifted

In [ ]:
shapes = sa_shifted.analyze_trajectory(
    trajectory=diff_trajectory,
    supercell=supercell,
    radius=radius,
)

for shape in shapes:
    fig = shape.plot(nbins=len(bins))
    fig.show()
    
    gemplt.shape(shape=shape,bins=bins)
    plt.show()

## Custom shifts

You can also use your own vectors to shift the sites:

In [ ]:
vectors = (
    [1, 0, 0],
    [2, 0, 0],
    [3, 0, 0],
)

sa_shifted = sa.shift_sites(vectors=vectors)
sa_shifted

## Get structure

Use `ShapeAnalyzer.get_structure()` to retrieve a pymatgen `Structure` object back:

In [ ]:
sa_shifted = sa_shifted.to_structure()
sa_shifted